Generation 과정 분산처리

In [1]:
import random


class Generation_model():
    def __init__(self, device) -> None:
        self.device = torch.device(f"cuda:{device}" if torch.cuda.is_available() else "cpu")
        self.model =  LlamaForCausalLM.from_pretrained("chavinlo/alpaca-native",low_cpu_mem_usage=True,torch_dtype='auto').to(self.device)
        self.tokenizer = LlamaTokenizer.from_pretrained("chavinlo/alpaca-native")
        self.device_number = device
    def generate(self,input_samples):
        candidate_list = []
        placeholders  = "'{}',"* 1
        formatted_prompt = f"prompt promp" + placeholders + "PROMPT PROMPT {}".format(input_samples) 
        print(input_samples)
        for input_sample in input_samples:
            prompt = formatted_prompt.format(*input_sample[1])
            inputs = self.tokenizer.encode(formatted_prompt,return_tensors="pt").to(self.device)
            candidate_senetnces = self.model.generate(inputs) #make to candidate format
            text = self.tokenizer.decode(*candidate_senetnces)
            candidate_list.append([input_sample[0],text])

        return candidate_list
        


In [1]:
from Generation.generation_model import Generation_model
from utils import *
import asyncio
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download
from PIL import Image
import torch
from transformers import AutoTokenizer, LlamaForCausalLM,LlamaTokenizer
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")
original_data = preprocess_dataset(dataset['train'])
few_data = list(make_fewshot_dataset(original_data,2).items())
sublists_length = int(len(few_data)/2)
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 446.46it/s]


In [2]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])

models = [Generation_model(device_number) for device_number in range(2)]
async_list = [generation_with_model_1(model) for model in models]

Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.99s/it]


In [4]:
async def generation_with_model_1(Generation_model = None):
    return Generation_model.generate(sublists[Generation_model.device_number])
    
    await asyncio.sleep(0)
async_list = [generation_with_model_1(model) for model in models]
async def main():
    results = await asyncio.gather(
        *async_list
    )
    return results
    


result = await main()


[('OBJECTIVE', ['To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ) .', 'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI .']), ('METHODS', ['This large , multicenter , simulated-use study evaluated whether adults ( aged 18-65 years ) , caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ) , and children ( aged 11-17 years ) , with and without experience in using an EAI , had a preference for the current design of Auvi-Q or the current design of EpiPen .', 'Forty patients were treated with RF and 40 with Cryo .'])]


/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[('RESULTS', ['We analysed the rates of stroke and systemic embolism in 6563 aspirin-treated patients with AF from the ACTIVE-A/AVERROES databases .', 'LPV/r-based ART was superior to NVP-based ART for efficacy and safety outcomes ; however , those on NVP had larger improvements in CD4 % , weight and height z-scores .']), ('CONCLUSIONS', ["OW individuals ' brain responses to food stimuli may indicate greater reward incentive processes than either SWL or NW groups .", 'Extended use of an alcohol-based mouthrinse is no more likely to cause reduction in salivary flow or perceived dryness in individuals with normal salivary flow compared with a non-alcohol-based mouthrinse ( CPH ) .'])]


In [5]:
print(result)

[[['OBJECTIVE', ' prompt promp\'{}\',PROMPT PROMPT [(\'OBJECTIVE\', [\'To assess the optimal scan delays and contrast injection durations for contrast-enhanced whole-body computed tomography ( CT ).\', \'The cross-sectional FOCUS ( Fixed-Dose Combination Drug for Secondary Cardiovascular Prevention ) study ( Phase 1 ) aimed to elucidate factors that interfere with appropriate adherence to CV medications for secondary prevention after an acute MI.\']), (\'METHODS\', [\'This large, multicenter, simulated-use study evaluated whether adults ( aged 18-65 years ), caregivers ( parents/guardians aged 18-65 years of children aged 5-17 years ), and children ( aged 11-17 years ), with and without experience in using an EAI, had a preference for the current design of Auvi-Q or the current design of EpiPen.\', \'Forty patients were treated with RF and 40 with Cryo.\'])]\n\ndef extract_paragraphs(paragraphs):\n    paragraphs_list = paragraphs.split("\\n\\n")\n    return'], ['METHODS', ' prompt prom

In [6]:
d = ['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
print(d)
placeholders  = "'{}',"* 1
formatted_prompt = "prompt promp" + placeholders + "PROMPT PROMPT {}" 
prompt = formatted_prompt.format(*d)
print(prompt)

['The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .', 'There was a weak correlation between prior contraceptive compliance and education level .']
prompt promp'The median time to prostate-specific antigen progression was 4.7 months ( 95 % confidence interval 3.7-8 .3 ) ; the median overall survival was 11.8 months .',PROMPT PROMPT There was a weak correlation between prior contraceptive compliance and education level .


In [7]:

async def generation_with_model_1(Generation_model = None):
    print(Generation_model.generate("hello"))
async_list = [generation_with_model_1(model) for model in models]
b=  generation_with_model_1(model1)
c =  generation_with_model_1(model2)
async def main():
    await asyncio.gather(
        b,c
    )
    print("finsh")


await main()

NameError: name 'model1' is not defined

In [4]:

await main()

TypeError: An asyncio.Future, a coroutine or an awaitable is required

In [15]:
async_list = []
for a in range(2):
    globals()[f"hell{a}"] = generation_with_model_1(a)
    async_list.append(f"hell{a}")

/tmp/ipykernel_2558908/2895284244.py:3: RuntimeWarning: coroutine 'generation_with_model_1' was never awaited
  globals()[f"hell{a}"] = generation_with_model_1(a)


'hell'

In [1]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

/home/nlp-04/anaconda3/envs/kkm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 469.55it/s]


In [52]:
labels = set()

# 유니크한 라벨 추출
for item in dataset['train']:
    label = item['label']
    labels.add(label)

labels = ['RESULTS','METHODS','OBJECTIVE','CONCLUSIONS']
# 라벨에 따라 요소를 분류할 딕셔너리 초기화
label_dict = {label: [] for label in labels}

# 리스트 순회하며 라벨에 따라 요소를 분류
for item in dataset['train']:
    label_dict[item['label']].append(item['text'])

# 결과 출력
for label, values in label_dict.items():
    print(label + ':', values)

KeyError: '###24854809'

In [45]:
label = list(set(get_labels(dataset)))

In [14]:
from datasets import load_dataset

dataset = load_dataset("ml4pubmed/pubmed-classification-20k")

Found cached dataset csv (/home/nlp-04/.cache/huggingface/datasets/ml4pubmed___csv/ml4pubmed--pubmed-classification-20k-3de662485795ef5d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 3/3 [00:00<00:00, 584.33it/s]


In [43]:
def preprocess_dataset(dataset):
# 레이블 정보 가져오기
    labels = dataset['label']  # 훈련 데이터셋의 레이블 가져오기

    # 레이블에 해당하는 인덱스 식별하기
    label_indices = {}
    for i, label in enumerate(labels):
        if label not in label_indices:
            label_indices[label] = []
        label_indices[label].append(i)

    # 레이블별로 데이터 저장할 리스트 생성
    label_lists = {}
    for label, indices in label_indices.items():
        if len(indices) > 1:  # 데이터가 있는 경우에만 리스트 생성
            label_lists[label] = []

    # 레이블별로 데이터 저장
    for label, indices in label_indices.items():
        if label in label_lists:  # 데이터가 있는 레이블에 대해서만 처리
            for index in indices:
                data = dataset[index]  # 데이터 가져오기 (여기서는 예시로 훈련 데이터셋 사용)
                label_lists[label].append(data['text'])
    return label_lists

In [48]:
b = preprocess_dataset(dataset['train'])

In [50]:
import random
def make_fewshot_dataset(label_lists,num_samples):
    fewshot_data = {}
    for label,all_data in label_lists.items():
        few_data = random.sample(all_data,num_samples)
        fewshot_data[label] = few_data
    return fewshot_data



In [66]:
few_data = list(make_fewshot_dataset(b,2).items())
sublists_length = 4 #class length / device lenght
sublists = [few_data[i:i+sublists_length] for i in range(0, len(few_data), sublists_length)]
sublists

[[('OBJECTIVE',
   ['To evaluate the efficacy of a combined neurolytic block of the celiac and superior hypogastric plexuses for incapacitating upper abdominal cancer pain .',
    'To investigate the role of preoperative biometry for selecting initial contact lens power .']),
  ('METHODS',
   ['Children aged 9-12years from both sexes were randomly assigned to one of six groups to receive plain milk , fortified milk , plain orange juice , fortified orange juice , supplement or placebo .',
    'Fasting blood samples were taken at baseline and after an 8-week intervention for quantification of related factors .']),
  ('RESULTS',
   ['No significant differences were found in the expression of sarcoplasmic reticulum Ca ( 2 + ) ATPase ( SERCA2a ) and ryanodine receptors ( RyRs ) .',
    'In an analysis controlling for level , degree , and number of anchors , tranexamic acid reduced drain output and total blood losses .']),
  ('CONCLUSIONS',
   ['However , there is a need for further research